User Segmentation Using RFM and debt/income/credit_score

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
DATA_PATH = Path.cwd().parent / 'merged-df.csv'
df = pd.read_csv(DATA_PATH)
print(f"Loaded dataset with {df.shape[0]:,} rows and {df.shape[1]:,} columns")

Loaded dataset with 13,305,915 rows and 36 columns


In [3]:
def _map_binary(series):
    truthy = {'y', 'yes', 'true', '1', 't'}
    falsy = {'n', 'no', 'false', '0', 'f'}
    s = series.astype(str).str.strip().str.lower()
    mapped = np.select([s.isin(truthy), s.isin(falsy)], [1, 0], default=np.nan)
    return pd.Series(mapped, index=series.index, dtype='float64')

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['card_acct_open_date'] = pd.to_datetime(df['card_acct_open_date'], errors='coerce')
df['card_expires'] = pd.to_datetime(df['card_expires'], errors='coerce')

df['use_chip_flag'] = _map_binary(df['use_chip']).fillna(0.0)
df['card_has_chip_flag'] = _map_binary(df['card_has_chip']).fillna(0.0)
df['card_on_dark_web_flag'] = _map_binary(df['card_card_on_dark_web']).fillna(0.0)

df['user_gender_code'], _ = pd.factorize(df['user_gender'])
df['card_brand_code'], _ = pd.factorize(df['card_card_brand'])
df['card_type_code'], _ = pd.factorize(df['card_card_type'])
df['merchant_state_code'], _ = pd.factorize(df['merchant_state'])
df['category_code'], _ = pd.factorize(df['category'])
df['mcc_code'], _ = pd.factorize(df['mcc'])

numeric_columns = [
    'user_current_age', 'user_retirement_age', 'user_birth_year', 'user_birth_month',
    'user_per_capita_income', 'user_yearly_income', 'user_total_debt',
    'user_credit_score', 'user_num_credit_cards', 'card_num_cards_issued',
    'card_credit_limit', 'card_year_pin_last_changed', 'user_latitude', 'user_longitude',
    'zip'
 ]
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
df['card_cvv'] = pd.to_numeric(df['card_cvv'], errors='coerce')

print("Preprocessing complete. Sample of engineered columns:")
df[['use_chip_flag', 'card_has_chip_flag', 'card_on_dark_web_flag', 'card_brand_code', 'card_type_code']].head()

Preprocessing complete. Sample of engineered columns:


,use_chip_flag,card_has_chip_flag,card_on_dark_web_flag,card_brand_code,card_type_code
0,0.0,1.0,0.0,0,0
1,0.0,1.0,0.0,0,1
2,0.0,1.0,0.0,0,2
3,0.0,0.0,0.0,0,2
4,0.0,1.0,0.0,1,2


In [4]:
df.columns

Index(['transaction_id', 'date', 'user_id', 'card_id', 'amount', 'use_chip',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc',
       'description', 'category', 'user_current_age', 'user_retirement_age',
       'user_birth_year', 'user_birth_month', 'user_gender', 'user_address',
       'user_latitude', 'user_longitude', 'user_per_capita_income',
       'user_yearly_income', 'user_total_debt', 'user_credit_score',
       'user_num_credit_cards', 'card_card_brand', 'card_card_type',
       'card_has_chip', 'card_cvv', 'card_expires', 'card_num_cards_issued',
       'card_credit_limit', 'card_acct_open_date',
       'card_year_pin_last_changed', 'card_card_on_dark_web', 'use_chip_flag',
       'card_has_chip_flag', 'card_on_dark_web_flag', 'user_gender_code',
       'card_brand_code', 'card_type_code', 'merchant_state_code',
       'category_code', 'mcc_code'],
      dtype='object')

In [5]:
df_rfm = df[['transaction_id', 'date', 'user_id', 'amount','merchant_city', 'user_gender']]
df_rfm.head()

# There is no missing values, I will start with RFM analysis

,transaction_id,date,user_id,amount,merchant_city,user_gender
0,7475327,2010-01-01 00:01:00,1556,-77.00,Beulah,Female
1,7475328,2010-01-01 00:02:00,561,14.57,Bettendorf,Male
2,7475329,2010-01-01 00:02:00,1129,80.00,Vista,Male
3,7475331,2010-01-01 00:05:00,430,200.00,Crown Point,Female
4,7475332,2010-01-01 00:06:00,848,46.41,Harwood,Male


In [6]:
def most_frequent(series):
    if series.empty:
        return np.nan
    mode = series.mode(dropna=True)
    return mode.iloc[0] if not mode.empty else np.nan

customer_agg = df.groupby('user_id').agg(
    first_txn_date=('date', 'min'),
    last_txn_date=('date', 'max'),
    txn_count=('transaction_id', 'count'),
    total_spent=('amount', 'sum'),
    avg_spent=('amount', 'mean'),
    max_spent=('amount', 'max'),
    merchant_count=('merchant_id', 'nunique'),
    merchant_city_count=('merchant_city', 'nunique'),
    merchant_state_count=('merchant_state', 'nunique'),
    merchant_zip_count=('zip', 'nunique'),
    mcc_count=('mcc', 'nunique'),
    category_count=('category', 'nunique'),
    unique_cards=('card_id', 'nunique'),
    card_credit_limit=('card_credit_limit', 'first'),
    card_year_pin_last_changed=('card_year_pin_last_changed', 'first'),
    card_acct_open_date=('card_acct_open_date', 'first'),
    card_expires=('card_expires', 'first'),
    user_current_age=('user_current_age', 'first'),
    user_retirement_age=('user_retirement_age', 'first'),
    user_gender_code=('user_gender_code', 'first'),
    user_per_capita_income=('user_per_capita_income', 'first'),
    user_yearly_income=('user_yearly_income', 'first'),
    user_total_debt=('user_total_debt', 'first'),
    user_credit_score=('user_credit_score', 'first'),
    user_num_credit_cards=('user_num_credit_cards', 'first'),
    most_frequent_city=('merchant_city', most_frequent),
    most_frequent_category=('category', most_frequent)
).reset_index()

customer_agg['first_txn_date'] = pd.to_datetime(customer_agg['first_txn_date'])
customer_agg['last_txn_date'] = pd.to_datetime(customer_agg['last_txn_date'])

reference_date = df['date'].max() + pd.Timedelta(days=1)

customer_agg['recency_days'] = (reference_date - customer_agg['last_txn_date']).dt.days
customer_agg['tenure_days'] = (customer_agg['last_txn_date'] - customer_agg['first_txn_date']).dt.days + 1
customer_agg.loc[customer_agg['tenure_days'] < 1, 'tenure_days'] = 1
customer_agg['txn_per_day'] = customer_agg['txn_count'] / customer_agg['tenure_days']

customer_agg['spend_per_txn'] = customer_agg['total_spent'] / customer_agg['txn_count'].replace(0, np.nan)
customer_agg['merchant_count_per_month'] = customer_agg['merchant_count'] / (customer_agg['tenure_days'] / 30.4375)
customer_agg['merchant_count_per_month'].replace([np.inf, -np.inf], np.nan, inplace=True)

customer_agg['card_account_age_years'] = ((reference_date - customer_agg['card_acct_open_date']).dt.days / 365.25)
customer_agg['card_account_age_years'] = customer_agg['card_account_age_years'].clip(lower=0)
customer_agg['card_time_to_expiry_years'] = ((customer_agg['card_expires'] - reference_date).dt.days / 365.25)
customer_agg['card_time_to_expiry_years'] = customer_agg['card_time_to_expiry_years'].fillna(0)

customer_agg['debt_to_income_ratio'] = customer_agg['user_total_debt'] / customer_agg['user_yearly_income'].replace(0, np.nan)
customer_agg['income_to_debt_ratio'] = customer_agg['user_yearly_income'] / customer_agg['user_total_debt'].replace(0, np.nan)
customer_agg['spend_to_limit_ratio'] = customer_agg['total_spent'] / customer_agg['card_credit_limit'].replace(0, np.nan)

customer_agg.replace([np.inf, -np.inf], np.nan, inplace=True)

customer_agg.head()

/tmp/nix-shell.M0b7RI/nix-shell.HFi73M/ipykernel_1141934/1656332588.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customer_agg['merchant_count_per_month'].replace([np.inf, -np.inf], np.nan, inplace=True)


,user_id,first_txn_date,last_txn_date,txn_count,total_spent,avg_spent,max_spent,merchant_count,merchant_city_count,merchant_state_count,...,recency_days,tenure_days,txn_per_day,spend_per_txn,merchant_count_per_month,card_account_age_years,card_time_to_expiry_years,debt_to_income_ratio,income_to_debt_ratio,spend_to_limit_ratio
0,0,2010-01-01 13:10:00,2019-10-31 19:14:00,12795,625799.67,48.909705,1128.47,518,288,43,...,1,3591,3.563074,48.909705,4.390595,12.167009,0.079398,0.607233,1.646813,35.556799
1,1,2010-01-01 11:58:00,2019-10-31 16:15:00,10073,336187.37,33.375099,937.15,189,76,21,...,1,3591,2.805068,33.375099,1.601974,12.167009,2.080767,0.321583,3.109618,26.264638
2,2,2010-01-01 06:47:00,2019-10-31 16:21:00,10612,291534.27,27.472132,519.02,146,37,10,...,1,3591,2.955166,27.472132,1.237503,14.417522,0.999316,2.945677,0.339481,39.396523
3,3,2010-01-02 17:15:00,2019-10-31 14:30:00,6001,280685.46,46.773114,990.20,162,66,23,...,1,3589,1.672053,46.773114,1.373885,12.670773,2.165640,0.668969,1.494838,4318.237846
4,4,2010-01-01 06:45:00,2019-10-31 21:26:00,15043,595722.36,39.601300,1624.15,449,187,37,...,1,3591,4.189084,39.601300,3.805747,10.417522,-5.672827,1.509361,0.662532,30.866444


In [7]:
# PCA analysis on customer metrics
numeric_features = [
    'recency_days', 'tenure_days', 'txn_count', 'total_spent', 'avg_spent', 'max_spent',
    'txn_per_day', 'spend_per_txn','merchant_count', 'merchant_city_count',
    'merchant_state_count', 'merchant_zip_count', 'mcc_count', 'category_count', 'unique_cards',
    'merchant_count_per_month', 'user_current_age', 'user_retirement_age', 
   'user_gender_code', 'user_per_capita_income', 'user_yearly_income',
    'user_total_debt', 'debt_to_income_ratio', 'income_to_debt_ratio', 'user_credit_score',
    'user_num_credit_cards',
    'card_credit_limit', 'card_year_pin_last_changed', 'card_account_age_years',
    'card_time_to_expiry_years', 'spend_to_limit_ratio'
 ]

feature_matrix = customer_agg[numeric_features].copy()
feature_matrix = feature_matrix.fillna(feature_matrix.median())

scaler = StandardScaler()
X_scaled = scaler.fit_transform(feature_matrix)

pca = PCA()
principal_components = pca.fit_transform(X_scaled)

explained_variance = pd.DataFrame({
    'principal_component': [f'PC{i+1}' for i in range(len(pca.explained_variance_ratio_))],
    'explained_variance_ratio': pca.explained_variance_ratio_,
    'cumulative_variance_ratio': np.cumsum(pca.explained_variance_ratio_)
})

pca_loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC{i+1}' for i in range(len(pca.components_))],
    index=numeric_features
)

important_features = pca_loadings['PC1'].abs().sort_values(ascending=False).rename('abs_loading_PC1')

explained_variance.head(10), pca_loadings.head(10), important_features.head(15)

(  principal_component  explained_variance_ratio  cumulative_variance_ratio
 0                 PC1                  0.252830                   0.252830
 1                 PC2                  0.117619                   0.370449
 2                 PC3                  0.069989                   0.440438
 3                 PC4                  0.054612                   0.495050
 4                 PC5                  0.050480                   0.545530
 5                 PC6                  0.043725                   0.589255
 6                 PC7                  0.039934                   0.629189
 7                 PC8                  0.038070                   0.667259
 8                 PC9                  0.035145                   0.702403
 9                PC10                  0.032235                   0.734639,
                           PC1       PC2       PC3       PC4       PC5  \
 recency_days        -0.028114  0.126539  0.021704 -0.314962  0.098368   
 tenure_days   

### Clustering workflow

In [8]:
n_selected = min(3, len(important_features))
top_features = important_features.head(n_selected).index.tolist()
print(f"Top features selected for clustering: {top_features}")

X_features = customer_agg[top_features]
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X_features)

k_values = range(2, 9)
TARGET_K = 4  # Set to None to pick k via silhouette score
kmeans = KMeans(n_clusters=TARGET_K , random_state=42, n_init='auto')
labels = kmeans.fit_predict(X_scaled)
customer_agg['Cluster'] = labels

Top features selected for clustering: ['merchant_zip_count', 'merchant_city_count', 'merchant_count']


### Cluster printing and export 

In [9]:
cluster_profile = customer_agg.groupby('Cluster').agg(
    customer_count=('user_id', 'nunique'),
    recency_days_mean=('recency_days', 'mean'),
    tenure_days_mean=('tenure_days', 'mean'),
    txn_count_mean=('txn_count', 'mean'),
    total_spent_mean=('total_spent', 'mean'),
).round(2).sort_index()

clustering_df = customer_agg[['user_id', 'Cluster']]
clustering_df.to_csv('clustering_df.csv', index=False)
print("Cluster assignments saved to clustering_df.csv")

cluster_profile

Cluster assignments saved to clustering_df.csv


,customer_count,recency_days_mean,tenure_days_mean,txn_count_mean,total_spent_mean
Cluster,,,,,
0,369,2.78,3568.27,12952.64,596540.59
1,108,1.01,3584.19,15274.09,819253.72
2,284,43.11,3306.62,7783.85,263847.45
3,458,10.96,3525.97,10188.16,411135.31
